In [5]:
import pandas as pd
import hashlib
import os
import uuid
import numpy as np
from IPython.display import clear_output

version = '1.12'
directory= "dataRaw/"

Chargement des données brutes

In [6]:


fileNames = ["2018-09-07-info5-info4-info3-Peip-Peip2.csv",
             "2018-10-16-To-2018-11-20-info5-info4-info3-Peip-Peip2.csv",
             "2018-01-22-info5.csv",
             "2017-Info5-22-01-2018.csv",
             "2018-10-25-To-2018-11-30-info5-info4-info3-Peip-Peip2.csv",
             "2018-11-05-To-2018-12-10-info5-info4-info3-Peip-Peip2-withObjectID.csv",
            "2018-11-12-To-2018-12-18-info5-info4-info3-Peip-Peip2-withObjectID.csv",
            "2018-12-02-To-2019-01-07-info5-info4-info3-Peip-Peip2-withObjectID.csv",
            "2018-12-23-To-2019-01-28-info5-info4-info3-Peip-Peip2-withObjectID.csv",
            "2019-02-03-To-2019-03-11-info5-info4-info3-Peip-Peip2-withObjectID.csv",
            "2019-07-31-To-2019-08-22-info5-info4-info3-Peip-Peip2-withObjectID.csv"]

dataRaws = []

for fileName in fileNames:
    dataRaw = pd.read_csv(os.path.join(directory,fileName), sep=',')

        
    if fileName in ["2018-01-22-info5.csv", "2017-Info5-22-01-2018.csv"]:
        dataRaw['event_timecreated'] = dataRaw['event_timecreated'].apply(lambda x: x/1000)
    
    if fileName not in ['2018-11-05-To-2018-12-10-info5-info4-info3-Peip-Peip2-withObjectID.csv']:
        dataRaw['object_id'] = 0
        dataRaw['context_id'] = 0
        
        dataRaw = dataRaw[['event_id','event_action','event_target',
                          'event_timecreated','object_id','context_id',
                          'course_id','course_fullname','course_shortname',
                          'course_newsitems','course_startdate','course_maxbytes',
                          'category_name','user_id','user_username','user_idnumber',
                          'user_firstname','user_lastname','user_email','user_lastlogin',
                          'user_currentlogin']]
    
    dataRaw['user_idnumber'] = dataRaw['user_idnumber'].astype(str)
    #dataRaw['user_id'] = dataRaw['user_id'].astype(int)   
    
    for column in ['event_timecreated', 'course_startdate', 'user_lastlogin', 'user_currentlogin']:
        dataRaw[column] = pd.to_numeric(dataRaw[column])
        dataRaw[column] = pd.to_datetime(dataRaw[column], unit='s')
    
    dataRaws.append(dataRaw)
    
dataRaw = pd.concat(dataRaws)    
dataRaw = dataRaw.reset_index()
dataRaw = dataRaw.drop(['index'], axis=1)
print("Shape dataRaw:"+str(dataRaw.shape))

dataRaw.drop_duplicates(subset=['event_id'], keep='first', inplace=True)

dataRaw['object_id'] = dataRaw['object_id'].replace([np.nan], 0)
dataRaw['object_id'] = dataRaw['object_id'].astype('int64')

dataRaw['user_id'] = dataRaw['user_id'].replace([np.nan], 0)
dataRaw['user_id'] = dataRaw['user_id'].astype('int64')

#dataRaw['user_id'] = dataRaw['user_id'].astype('int64')
#df.replace([np.inf, -np.inf], np.nan)

print("Shape dataRaw:"+str(dataRaw.shape))
print("types:"+str(dataRaw.dtypes))

#print("values:"+str(dataRaw['user_id'].unique()))

print()
dataRaw.head()

Shape dataRaw:(387353, 21)
Shape dataRaw:(250135, 21)
types:event_id                      int64
event_action                 object
event_target                 object
event_timecreated    datetime64[ns]
object_id                     int64
context_id                    int64
course_id                     int64
course_fullname              object
course_shortname             object
course_newsitems              int64
course_startdate     datetime64[ns]
course_maxbytes               int64
category_name                object
user_id                       int64
user_username                object
user_idnumber                object
user_firstname               object
user_lastname                object
user_email                   object
user_lastlogin       datetime64[ns]
user_currentlogin    datetime64[ns]
dtype: object



,event_id,event_action,event_target,event_timecreated,object_id,context_id,course_id,course_fullname,course_shortname,course_newsitems,...,course_maxbytes,category_name,user_id,user_username,user_idnumber,user_firstname,user_lastname,user_email,user_lastlogin,user_currentlogin
0,205324505,viewed,course,2018-08-12 15:24:51,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,...,209715200,info3,191079,e155283l,e155283l,Antoine,BOURSIN,antoine.boursin@etu.univ-nantes.fr,2018-09-06 08:18:55,2018-09-06 15:55:01
1,205699617,viewed,course,2018-08-16 10:58:27,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,...,209715200,info3,185748,e157371f,e157371f,Alexandre,NONNON,alexandre.nonnon@etu.univ-nantes.fr,2018-09-06 11:12:42,2018-09-06 15:29:52
2,206188017,created,user_enrolment,2018-08-21 07:46:07,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,...,209715200,info3,2,admin,admin,Moodle,ADMIN,Moodle.Admin@univ-nantes.fr,2010-01-13 14:58:47,2010-09-01 08:49:59
3,206188018,assigned,role,2018-08-21 07:46:07,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,...,209715200,info3,2,admin,admin,Moodle,ADMIN,Moodle.Admin@univ-nantes.fr,2010-01-13 14:58:47,2010-09-01 08:49:59
4,206246728,viewed,course,2018-08-21 18:10:01,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,...,209715200,info3,233714,e179023q,e179023q,Johan,SORETTE,johan.sorette@etu.univ-nantes.fr,2018-08-24 04:54:17,2018-08-29 18:22:27


Ajout données avec nouvelle colonne :

In [7]:
dataRaw.iloc[:, [17]]

,user_lastname
0,BOURSIN
1,NONNON
2,ADMIN
3,ADMIN
4,SORETTE
5,ADMIN
6,ADMIN
7,ADMIN
8,ADMIN
9,ADMIN


Gestion encodage :

In [8]:
dataRaw['course_fullname'] = dataRaw['course_fullname'].astype(str)
#dataRaw['course_fullname'] = dataRaw['course_fullname'].str.encode(encoding='utf-8')

#def setDecode(row):
    
#    name = bytes(row['course_fullname'])
    
#    return name.decode("utf-8") 

#dataRaw['course_fullname'] = dataRaw.apply(lambda row: setDecode(row),axis=1)


dataRaw['course_fullname']

0         Info3 - Systèmes à Base de Connaissances
1         Info3 - Systèmes à Base de Connaissances
2         Info3 - Systèmes à Base de Connaissances
3         Info3 - Systèmes à Base de Connaissances
4         Info3 - Systèmes à Base de Connaissances
5         Info3 - Systèmes à Base de Connaissances
6         Info3 - Systèmes à Base de Connaissances
7         Info3 - Systèmes à Base de Connaissances
8         Info3 - Systèmes à Base de Connaissances
9         Info3 - Systèmes à Base de Connaissances
10        Info3 - Systèmes à Base de Connaissances
11        Info3 - Systèmes à Base de Connaissances
12        Info3 - Systèmes à Base de Connaissances
13        Info3 - Systèmes à Base de Connaissances
14        Info3 - Systèmes à Base de Connaissances
15        Info3 - Systèmes à Base de Connaissances
16        Info3 - Systèmes à Base de Connaissances
17        Info3 - Systèmes à Base de Connaissances
18        Info3 - Systèmes à Base de Connaissances
19        Info3 - Systèmes à Ba

Quelques statistiques:

In [9]:
print("lignes x colonnes:"+str(dataRaw.shape))

print("Promotion:"+str(dataRaw['category_name'].unique()))

lignes x colonnes:(250135, 21)
Promotion:['info3' 'info4' 'info5' 'PEIP' 'PEIP 2'
 'Semestre 7 (septembre -> janvier)' 'Semestre 5 (septembre -> janvier)'
 'INFORMATIQUE' 'Semestre 6 (janvier -> juin)'
 'Semestre 8 (janvier -> juin)' 'PEIP A 1' 'PEIP A 2' 'PEIP D 1'
 'PEIP D 2']


Détection enseignant & admin:

In [10]:
def setType(row):
    
    email = str(row['user_email'])
    
    if email == 'Moodle.Admin@univ-nantes.fr':
        return 'Admin'
    elif email == 'root@localhost':
        return 'Invité'
    elif "@univ-nantes.fr" in email:
        return 'Enseignant'
    elif "@etu.univ-nantes.fr" in email:
        return 'Etudiant'
    else:
        return 'Indéfini'

#dataRaw[dataRaw['user_idnumber'] == 'admin'].describe()

dataRaw['type'] = dataRaw.apply(lambda row: setType(row),axis=1)

dataRaw.head()

,event_id,event_action,event_target,event_timecreated,object_id,context_id,course_id,course_fullname,course_shortname,course_newsitems,...,category_name,user_id,user_username,user_idnumber,user_firstname,user_lastname,user_email,user_lastlogin,user_currentlogin,type
0,205324505,viewed,course,2018-08-12 15:24:51,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,...,info3,191079,e155283l,e155283l,Antoine,BOURSIN,antoine.boursin@etu.univ-nantes.fr,2018-09-06 08:18:55,2018-09-06 15:55:01,Etudiant
1,205699617,viewed,course,2018-08-16 10:58:27,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,...,info3,185748,e157371f,e157371f,Alexandre,NONNON,alexandre.nonnon@etu.univ-nantes.fr,2018-09-06 11:12:42,2018-09-06 15:29:52,Etudiant
2,206188017,created,user_enrolment,2018-08-21 07:46:07,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,...,info3,2,admin,admin,Moodle,ADMIN,Moodle.Admin@univ-nantes.fr,2010-01-13 14:58:47,2010-09-01 08:49:59,Admin
3,206188018,assigned,role,2018-08-21 07:46:07,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,...,info3,2,admin,admin,Moodle,ADMIN,Moodle.Admin@univ-nantes.fr,2010-01-13 14:58:47,2010-09-01 08:49:59,Admin
4,206246728,viewed,course,2018-08-21 18:10:01,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,...,info3,233714,e179023q,e179023q,Johan,SORETTE,johan.sorette@etu.univ-nantes.fr,2018-08-24 04:54:17,2018-08-29 18:22:27,Etudiant


Rename the category name :

In [11]:
dataRaw.category_name.replace('Semestre 5 (septembre -> janvier)', 'info3', inplace=True)
dataRaw.category_name.replace('Semestre 6 (janvier -> juin)', 'info3', inplace=True)
dataRaw.category_name.replace('Semestre 7 (septembre -> janvier)', 'info4',  inplace=True)
dataRaw.category_name.replace('Semestre 8 (janvier -> juin)', 'info4',  inplace=True)

print("lignes x colonnes:"+str(dataRaw.shape))

print("Promotion:"+str(dataRaw['category_name'].unique()))
#dataRaw.category_name.replace(to_replace=dict('Semestre 7 (septembre -> janvier)'='info4', 
#                                              'Semestre 8 (janvier -> juin)'='info4'), inplace=True)


lignes x colonnes:(250135, 22)
Promotion:['info3' 'info4' 'info5' 'PEIP' 'PEIP 2' 'INFORMATIQUE' 'PEIP A 1'
 'PEIP A 2' 'PEIP D 1' 'PEIP D 2']


Création colonne pour anonymisation :

In [12]:
dataAnonym = pd.DataFrame(data=dataRaw[['user_username', 'user_idnumber', 'user_firstname', 'user_lastname', 'user_email']])
dataAnonym = dataAnonym.drop_duplicates()

print("Nombre d'utilisateurs:"+str(dataAnonym.shape[0]))
dataAnonym.head()

Nombre d'utilisateurs:605


,user_username,user_idnumber,user_firstname,user_lastname,user_email
0,e155283l,e155283l,Antoine,BOURSIN,antoine.boursin@etu.univ-nantes.fr
1,e157371f,e157371f,Alexandre,NONNON,alexandre.nonnon@etu.univ-nantes.fr
2,admin,admin,Moodle,ADMIN,Moodle.Admin@univ-nantes.fr
4,e179023q,e179023q,Johan,SORETTE,johan.sorette@etu.univ-nantes.fr
271,e165359n,e165359n,Killian,COUTY,killian.couty@etu.univ-nantes.fr


Hashcode des user_id:

In [13]:

#funcHash = lambda id: hashlib.md5(id.encode()+code.encode()).hexdigest()
#serie = dataAnonym['user_idnumber'].apply()
#dataAnonym['id_hash'] = serie
#print(serie.head())
#print("serie shape:"+str(serie.shape))
#print("dataAnonym shape:"+str(dataAnonym.shape))

dataAnonym['uuid'] = dataAnonym.apply(lambda x: str(uuid.uuid4()), axis=1)

dataAnonym['uuid'] = dataAnonym['uuid'].astype(str)

dataAnonym.head()


,user_username,user_idnumber,user_firstname,user_lastname,user_email,uuid
0,e155283l,e155283l,Antoine,BOURSIN,antoine.boursin@etu.univ-nantes.fr,3c52e7e5-d51b-49f0-9d1a-2f3483121a9b
1,e157371f,e157371f,Alexandre,NONNON,alexandre.nonnon@etu.univ-nantes.fr,16b148fe-38d8-41f1-8203-39e2faa35ee8
2,admin,admin,Moodle,ADMIN,Moodle.Admin@univ-nantes.fr,d6085ec6-a498-4d28-9d39-3f1804903dfd
4,e179023q,e179023q,Johan,SORETTE,johan.sorette@etu.univ-nantes.fr,e2cdd600-23fb-443e-bf0e-0a2c7c50a22e
271,e165359n,e165359n,Killian,COUTY,killian.couty@etu.univ-nantes.fr,434d2401-a111-408d-9e5d-53f56f61cfbb


Suppression des colonnes de dataRaw:

In [14]:
data = dataRaw.drop(['user_username', 'user_firstname', 'user_lastname', 'user_email', 'user_id'], axis=1)

print("data shape:"+str(data.shape))

data.head()

data shape:(250135, 17)


,event_id,event_action,event_target,event_timecreated,object_id,context_id,course_id,course_fullname,course_shortname,course_newsitems,course_startdate,course_maxbytes,category_name,user_idnumber,user_lastlogin,user_currentlogin,type
0,205324505,viewed,course,2018-08-12 15:24:51,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,2007-09-21 22:00:00,209715200,info3,e155283l,2018-09-06 08:18:55,2018-09-06 15:55:01,Etudiant
1,205699617,viewed,course,2018-08-16 10:58:27,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,2007-09-21 22:00:00,209715200,info3,e157371f,2018-09-06 11:12:42,2018-09-06 15:29:52,Etudiant
2,206188017,created,user_enrolment,2018-08-21 07:46:07,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,2007-09-21 22:00:00,209715200,info3,admin,2010-01-13 14:58:47,2010-09-01 08:49:59,Admin
3,206188018,assigned,role,2018-08-21 07:46:07,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,2007-09-21 22:00:00,209715200,info3,admin,2010-01-13 14:58:47,2010-09-01 08:49:59,Admin
4,206246728,viewed,course,2018-08-21 18:10:01,0,0,14951,Info3 - Systèmes à Base de Connaissances,ipolytechbc3,5,2007-09-21 22:00:00,209715200,info3,e179023q,2018-08-24 04:54:17,2018-08-29 18:22:27,Etudiant


Fusion des deux data frames:

In [15]:
dataFinal = data.merge(dataAnonym[['user_idnumber', 'uuid']], on='user_idnumber')
dataFinal = dataFinal.drop('user_idnumber', axis=1)

dataFinal.drop_duplicates(subset=['event_id'],  keep='first', inplace=True)

print("Pas de doublon dans les lignes:"+str(dataFinal.duplicated().unique()[0] == False))

dataFinal = dataFinal.sort_values(by=['event_timecreated'])
dataFinal['user_idnumber'] = dataFinal['uuid']

dataFinal.to_csv("DataReady/madoc."+version+".csv.gz", index=False, compression='gzip', encoding='utf-8')
dataAnonym.to_csv("DataReady/anonym."+version+".csv", index=False)

dataFinal.head()

Pas de doublon dans les lignes:True


,event_id,event_action,event_target,event_timecreated,object_id,context_id,course_id,course_fullname,course_shortname,course_newsitems,course_startdate,course_maxbytes,category_name,user_lastlogin,user_currentlogin,type,uuid,user_idnumber
247679,152835032,viewed,course,2017-12-17 08:42:49,0,0,15159,INFO5- S9 - ID - Aide multicritère à la décision,ipolytecham,5,2007-09-27 22:00:00,41943040,info5,2018-01-19 16:54:18,2018-01-21 17:13:04,Etudiant,654c72a9-2585-4771-89f9-cca2473d7810,654c72a9-2585-4771-89f9-cca2473d7810
247680,152835060,viewed,course_module,2017-12-17 08:43:27,0,0,15159,INFO5- S9 - ID - Aide multicritère à la décision,ipolytecham,5,2007-09-27 22:00:00,41943040,info5,2018-01-19 16:54:18,2018-01-21 17:13:04,Etudiant,654c72a9-2585-4771-89f9-cca2473d7810,654c72a9-2585-4771-89f9-cca2473d7810
247681,152835084,viewed,course_module,2017-12-17 08:43:59,0,0,15159,INFO5- S9 - ID - Aide multicritère à la décision,ipolytecham,5,2007-09-27 22:00:00,41943040,info5,2018-01-19 16:54:18,2018-01-21 17:13:04,Etudiant,654c72a9-2585-4771-89f9-cca2473d7810,654c72a9-2585-4771-89f9-cca2473d7810
246742,152835097,viewed,course,2017-12-17 08:44:10,0,0,24600,Projet de recherche et développement,ipolytechi5prd,5,2009-09-08 22:00:00,41943040,info5,2018-01-21 14:32:40,2018-01-21 20:05:03,Etudiant,8efdb168-459e-4593-86e5-5e4bedb90c60,8efdb168-459e-4593-86e5-5e4bedb90c60
247682,152835099,viewed,course_module,2017-12-17 08:44:12,0,0,15159,INFO5- S9 - ID - Aide multicritère à la décision,ipolytecham,5,2007-09-27 22:00:00,41943040,info5,2018-01-19 16:54:18,2018-01-21 17:13:04,Etudiant,654c72a9-2585-4771-89f9-cca2473d7810,654c72a9-2585-4771-89f9-cca2473d7810


Test anonymisation:

In [16]:
# e157371f	

print("dataRaw e157371f: "+str(dataRaw[dataRaw['user_idnumber'] == 'e157371f'].shape))

uuidEtu = dataAnonym[dataAnonym['user_idnumber'] == 'e157371f']['uuid'].values[0]

print("\nuuid Etu:"+str(uuidEtu))

print("hash of e157371f: "+str(dataAnonym[dataAnonym['user_idnumber'] == 'e157371f']['uuid']))

print("\n dataRaw e157371f: "+str(dataFinal[dataFinal['uuid'] == uuidEtu].shape))

print("Test is "+str(dataRaw[dataRaw['user_idnumber'] == 'e157371f'].shape[0] == dataFinal[dataFinal['uuid'] == uuidEtu].shape[0]))

dataRaw e157371f: (172, 22)

uuid Etu:16b148fe-38d8-41f1-8203-39e2faa35ee8
hash of e157371f: 1    16b148fe-38d8-41f1-8203-39e2faa35ee8
Name: uuid, dtype: object

 dataRaw e157371f: (172, 18)
Test is True
